# Galina Brusova

# [SF-DST] Credit Scoring_v.1.0


### Импорт библиотек

In [ ]:
import warnings
from pandas import Series
import pandas as pd
import numpy as np
import collections

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import recall_score, precision_score, f1_score, log_loss
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

import matplotlib
plt.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 8)

pd.set_option('display.max_rows', 50)  # показывать больше строк
pd.set_option('display.max_columns', 50)  # показывать больше колонок

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

warnings.filterwarnings("ignore")  # Убирает некритические сообщения

### Загрузка и изучение датасета

In [ ]:
train = pd.read_csv('/kaggle/input/sf-dst-scoring/train.csv')
test= pd.read_csv('/kaggle/input/sf-dst-scoring/test.csv')
sample_submission = pd.read_csv('/kaggle/input/sf-dst-scoring/sample_submission.csv')

In [ ]:
pd.set_option('display.max_columns', None)
print('Размерность тренировочного датасета: ', train.shape)
display(train.head(3))
print('Размерность тестового датасета: ', test.shape)
display(test.head(3))
print('Размерность sample_submission: ', sample_submission.shape)
display(sample_submission.head(3))

In [ ]:
RANDOM_SEED = 42
!pip freeze > requirements.txt

In [ ]:
train.columns

In [ ]:
test.columns

Описания полей датасета

- client_id - идентификатор клиента
- education - уровень образования
- sex - пол заемщика
- age - возраст заемщика
- car - флаг наличия автомобиля
- car_type - флаг автомобиля иномарки
- decline_app_cnt - количество отказанных прошлых заявок
- good_work - флаг наличия “хорошей” работы
- bki_request_cnt - количество запросов в БКИ
- home_address - категоризатор домашнего адреса
- work_address - категоризатор рабочего адреса
- income - доход заемщика
- foreign_passport - наличие загранпаспорта
- sna - связь заемщика с клиентами банка
- first_time - давность наличия информации о заемщике
- score_bki - скоринговый балл по данным из БКИ
- region_rating - рейтинг региона
- app_date - дата подачи заявки
- default - флаг дефолта по кредиту

In [ ]:
display (train)

In [ ]:
train.info()

In [ ]:
#первый взгляд
import pandas_profiling
#pandas_profiling.ProfileReport(train)

In [ ]:
sns.countplot(x='default', data=train)

In [ ]:
train_default = train[train.default == 1]
train = pd.concat([train, train_default, train_default, train_default, train_default, train_default, train_default])
sns.countplot(x='default', data=train)

In [ ]:
test['default'] = -1

In [ ]:
sns.countplot(x='default', data=train)

In [ ]:
train.info()

In [ ]:
test.info()

## Заменим дату на два новых признака: месяц и год 


In [ ]:
def year(i):
     return int(i[-4:]) 
train['year'] = train['app_date'].apply(year)

def season(k):
     return str(k[-7:-4])   
train['season'] = train['app_date'].apply(season)

def year_test(i):
     return int(i[-4:]) 
test['year'] = test['app_date'].apply(year_test)
def season_test(k):
     return str(k[-7:-4])   
test['season'] = test['app_date'].apply(season_test)

In [ ]:
del train['app_date']
del test['app_date']
display(train)
display(test)

In [ ]:
print(train.season.unique())
print(train.year.unique())

## Работа с пропусками

In [ ]:
train.isnull().sum()

## Посмотрим на уникальные значения в education и удалим пропуски:

In [ ]:
print(train.education.unique())
print(test.education.unique())
print(train.education.value_counts())
print(test.education.value_counts())

In [ ]:
## функция замены значений на наиболее часто встречающиеся
def change_max_train(column):
    # присвоим значениям NaN самое частое значение
    return train[column].fillna((train[column].value_counts().idxmax()), inplace=True)
def change_max_test(column):
    # присвоим значениям NaN самое частое значение
    return test[column].fillna((test[column].value_counts().idxmax()), inplace=True)

In [ ]:
change_max_test('education')
change_max_train('education')
# проверим результат
print(train.education.unique())
print(test.education.unique())

## Разобьем переменные на категории

In [ ]:
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']
cat_cols = ['education', 'home_address', 'work_address', 'sna', 'first_time','region_rating','season']
num_cols = ['age', 'decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income']

## Рассмотрим числовые признаки

In [ ]:
for i in num_cols:
    plt.figure()
    sns.distplot(train[i][train[i] > 0].dropna(), kde = False, rug=False)
    plt.title(i)
    plt.show()

## Построим боксплоты: 

In [ ]:
# Checking the frequency distribution.
fig, axes = plt.subplots(2, 3, figsize=(15, 15))
axes = axes.flatten()
for i in range(len(num_cols)):
    sns.boxplot(x="default", y=num_cols[i], data=train, ax=axes[i])

In [ ]:
def get_boxplot(column):
    fig, ax = plt.subplots(figsize = (14, 4))
    sns.boxplot(x=column, y='age', 
                data=train.loc[train.loc[:, column].isin(train.loc[:, column].value_counts().index[:10])],
               ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()

In [ ]:
for col in ['decline_app_cnt','income','bki_request_cnt']:
    get_boxplot(col)

## Построим графики распределения логарифмированных переменных.

In [ ]:
sns.heatmap(train.corr().abs(), vmin=0, vmax=1)

In [ ]:
train.corr()

In [ ]:
corr_cols = num_cols + ['default']
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap((train[corr_cols]).corr(),cmap='coolwarm', annot=True)

#### можно отметить, что на дефолт по кредиту влияет скоринг БКИ и предыдущие отказы

In [ ]:
train.isnull().sum()
test.isnull().sum()

## Посмотрим на уникальные значение признаков и их количество

In [ ]:
train.agg({'nunique', lambda x: x.unique()}).transpose()

## Посмотрим значимость непрерывных переменных

In [ ]:
imp_num = Series(f_classif(train[num_cols], train['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

## Категориальные переменные

In [ ]:
label_encoder = LabelEncoder()

for column in bin_cols:
    train[column] = label_encoder.fit_transform(train[column])
    test[column] = label_encoder.fit_transform(test[column])
print(dict(enumerate(label_encoder.classes_)))

In [ ]:
label_encoder = LabelEncoder()

mapped_education = pd.Series(label_encoder.fit_transform(train['season']))
print(dict(enumerate(label_encoder.classes_)))

In [ ]:
label_encoder = LabelEncoder()

mapped_education = pd.Series(label_encoder.fit_transform(test['season']))
print(dict(enumerate(label_encoder.classes_)))

In [ ]:
# Для бинарных признаков мы будем использовать LabelEncoder

label_encoder = LabelEncoder()

for column in bin_cols:
    train[column] = label_encoder.fit_transform(train[column])
    
# убедимся в преобразовании    
train.head()
#test.head()

In [ ]:
label_encoder = LabelEncoder()

mapped_education = pd.Series(label_encoder.fit_transform(train['education']))
print(dict(enumerate(label_encoder.classes_)))

In [ ]:
# Для бинарных признаков мы будем использовать LabelEncoder. Train

label_encoder = LabelEncoder()

for column in cat_cols:
    train[column] = label_encoder.fit_transform(train[column])
    
# убедимся в преобразовании    
train.head()

In [ ]:
# Для бинарных признаков мы будем использовать LabelEncoder. Test

label_encoder = LabelEncoder()

for column in cat_cols:
    test[column] = label_encoder.fit_transform(test[column])
    
# убедимся в преобразовании    
test.head()

In [ ]:
#kategorial
fig,axes = plt.subplots(2, 4, figsize=(15,10))
plt.subplots_adjust(wspace=0.5)
axes = axes.flatten()
for i in range(len(cat_cols)):
    sns.countplot(x=cat_cols[i], data=train, ax=axes[i])

In [ ]:
# Функция определяет межквартильный интервал и возвращает 1.5 межквартильных расстояния с обеих сторон от этого интервала. С её помощью избавимся от выбросов.
def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return lower_bound, upper_bound

In [ ]:
del test['default']

## Добавим новый признак rait, рейтинг на основе признаков благосостояния

In [ ]:
test['rait'] = test['car'] + test['car_type'] +  test['foreign_passport'] 
train['rait'] = train['car'] + train['car_type'] + train['foreign_passport'] 

## Проверим влияние нового параметра

In [ ]:
num_cols = ['age', 'decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income','rait']
imp_num = Series(f_classif(train[num_cols], train['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [ ]:
num_cols = ['age', 'decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income']

In [ ]:
train

## Посмотрим значимость категориальных и бинарных переменных

In [ ]:
imp_cat = pd.Series(mutual_info_classif(train[bin_cols + cat_cols], train['default'],
                                     discrete_features =True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

# Преобразуем категориальные переменные при помощи OneHotEncoder

In [ ]:
train.isnull().sum(), test.isnull().sum()

In [ ]:
cat_cols = ['education', 'home_address', 'work_address', 'sna', 'first_time','region_rating']
display
x_cat = OneHotEncoder(sparse=False).fit_transform(train[cat_cols].values)
y_cat = OneHotEncoder(sparse=False).fit_transform(test[cat_cols].values)

print(x_cat.shape)
print(y_cat.shape)

# Подготовка к машинному обучению

In [ ]:
poly = PolynomialFeatures(2)

x_p = poly.fit_transform(train[num_cols].values)
y_p = poly.fit_transform(test[num_cols].values)

In [ ]:
# Стандартизация числовых переменных

X_num = StandardScaler().fit_transform(x_p)
X_num.shape

In [ ]:
Y_num = StandardScaler().fit_transform(y_p)
Y_num.shape

In [ ]:
print(X_num)
print(Y_num)

In [ ]:
train

In [ ]:
X = np.hstack([X_num, train[bin_cols].values, x_cat])
Y = train['default'].values

id_test = test['client_id']
test = np.hstack([Y_num, test[bin_cols].values, y_cat])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.20,
                                                    random_state=RANDOM_SEED,
                                                    shuffle=True)

## Подбор гиперпараметров (регуляризация)

In [ ]:
# Зададим ограничения для параметра регуляризации
C = np.logspace(0, 4, 10)

penalty = ['l1', 'l2']
hyperparameters = dict(C=C, penalty=penalty)

model = LogisticRegression()
model.fit(X_train, y_train)

clf = GridSearchCV(model, hyperparameters, cv=5, verbose=0)

best_model = clf.fit(X_train, y_train)

print('Лучший penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Лучшее C:', best_model.best_estimator_.get_params()['C'])

In [ ]:
#from sklearn.model_selection import GridSearchCV

# Добавим типы регуляризации
#penalty = ['l1', 'l2']

# Зададим ограничения для параметра регуляризации
#C = np.logspace(0, 4, 10)

# Создадим гиперпараметры
#hyperparameters = dict(C=C, penalty=penalty)

#model = LogisticRegression()
#model.fit(X_train, y_train)

# Создаем сетку поиска с использованием 5-кратной перекрестной проверки
#clf = GridSearchCV(model, hyperparameters, cv=5, verbose=0)

#best_model = clf.fit(X_train, y_train)

# View best hyperparameters
#print('Лучшее Penalty:', best_model.best_estimator_.get_params()['penalty'])
#print('Лучшее C:', best_model.best_estimator_.get_params()['C'])

## Обучение и метрики

In [ ]:
# Обучим модель

model = LogisticRegression(penalty='l2', C=7.742636826811269, max_iter=800)
model.fit(X_train, y_train)

In [ ]:
probs = model.predict_proba(X_test)
probs = probs[:, 1]


fpr, tpr, threshold = roc_curve(y_test, probs)
roc_auc = roc_auc_score(y_test, probs)

# Визуализация ROC AUC
plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label='Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Функция выводит метрики accuracy и f1-score
def print_logisitc_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f'acc = {acc:.2f} F1-score = {f1:.2f}')

In [ ]:
Y_predicted = model.predict(X_test)
print_logisitc_metrics(y_test, Y_predicted)

In [ ]:
# confusion matrix
cm = confusion_matrix(y_test, Y_predicted)
cm

In [ ]:
# визуализация confusion matrix
sns.set_context(context='paper', font_scale=2, rc=None)
group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
group_counts = ['{0:0.0f}'.format(value) for value in
                cm.flatten()]
labels = [f'{v1}\n{v2}' for v1, v2 in
          zip(group_names, group_counts)]
labels = np.asarray(labels).reshape(2, 2)
sns.heatmap(cm, annot=labels, fmt='', cmap='Blues')

In [ ]:
model = LogisticRegression(penalty='l2', C=7.742636826811269, max_iter=800)
model.fit(X, Y)
probs = model.predict_proba(test)
probs = probs[:, 1]

In [ ]:
my_submission = pd.DataFrame({'client_id': id_test,
                              'default': probs})
my_submission.to_csv('submission.csv', index=False)

my_submission.head(10)